## 3.3. Concise Implementation of Linear Regression

Broad and intense interest in deep learning for the past several years has inspired companies, academics and hobbyists to develop a variety of mature open source frameworks for automating the repetitive work of implementing gradient-based learning algorithms. In Section 3.2, we relied only on (1) tensors for data storage and linear algebra, and (2) auto-differentiation for calculating gradients. In practice, because data iterators, loss functions, optimisers and neural network layers are so common, modern libraries implement these components for us as well.

In this section, we will implement the linear regression model from Section 3.2 concisely by using high-level APIs of deep learning frameworks.

### 3.3.1. Generating the Dataset

To start, we will generate the same dataset as in Section 3.2.

In [32]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

### 3.3.2. Reading the Dataset

Rather than rolling our own iterator, we can call upon the existin API in a framework to read data. We pass in `features` and `labels` as arguments and specify `batch_size` when instantiating a data iterator object. Besides, the boolean value `is_train` indicates whether or not we want the data iterator object to shuffle the data on each epoch (pass through the dataset).

In [33]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

Now we can use `data_iter` in much the same way as we called the `data_iter` function in Section 3.2. To verify that it is working, we can read and print the first minibatch of examples. Comparing with Section 3.2, here we use `iter` to construct a Python iterator and use `next` to obtain the first item from the iterator.

In [34]:
next(iter(data_iter))

[tensor([[-0.6154,  1.6685],
         [-0.4393,  1.0461],
         [ 1.3940,  0.4749],
         [-0.0440, -0.2620],
         [-0.8406, -0.7018],
         [ 0.5724,  0.3277],
         [-1.7779, -1.5686],
         [ 0.5438, -0.7213],
         [-0.0809,  2.0114],
         [ 1.1757,  0.4219]]),
 tensor([[-2.7061],
         [-0.2285],
         [ 5.3716],
         [ 5.0151],
         [ 4.9220],
         [ 4.2401],
         [ 5.9846],
         [ 7.7581],
         [-2.8045],
         [ 5.1219]])]

### 3.3.3. Defining the Model

When we implemented linear regression from scratch in Section 3.2, we defined our model parameters and coded up the calculations to produce output using basic linear algebra expressions. 

For standard operations, we can use a framework's predefined layers, which allow us to focus especially on the layers used to construct the model rather than having to focus on the implementation. 
we will first define a model variable `net`, which will refer to an instance of the `Sequential` class. The `Sequential` class defines a container for several layers that will be chained together. Given input data, a `Sequential` instance passes it through the first layer, in turn passing the output as the second layer's input and so forth. In the following example, our model consists of only one layer, so we do not really need `Sequential`. But since nearly all of our future models will involve multiple layers, we will use it anyway just to familiarise ourselves with the most standard workflow.

Recall the architecture of a single-layer network as shown in Figure 3.1.2. The layer is said to be *fully-connected* because each of its inputs is connected to each of its outputs by means of a matrix-vector multiplication.

In [35]:
# `nn` is an abbreviation for neural networks
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

### 3.3.4. Initialising Model Parameters

Before using `net`, we need to initialise the model parameters, such as the weights and bias in the linear regression model. Deep learning frameworks oftwn have a predefined way to initialise the parameters. Here we specify that each weight parameter should be randomly sampled from a normal distribution with mean 0 and standard deviation 0.01. The bias parameter will be initialised to 0.

As we have specified the input and output dimensions when constructing `nn.Linear`, now we can access the parameters directly to specify their initial values. We first locate the layer by `net[0]`, which is the first layer of the network, and use the `weight.data` and `bias.data` methods to access the parameters. Next we use the replace methods `normal_` and `fill_` to overwrite parameter values.

In [36]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

### 3.3.5. Defining the Loss Function

The `MSELoss` class computes the mean squared error (without the $1/2$ factor in Equation 3.1.5). By default it returns the average loss over examples.

In [37]:
loss = nn.MSELoss()

Minibatch stochastic gradient descent is a standard tool for optimising neural networks and thus PyTorch supports  it alongside a number of variations on this algorithm in the `optim` module. When we instantiate an `SGD` instance, we will specify the parameters to optimise over (obtainable from `net` via `net.parameters()`), with a dictionary of hyperparameters required by our optimisation algorithm. Minibatch stochastic gradient descent just requires that we set the value `lr`, which we set to 0.03 here.

In [38]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

### 3.3.7. Training

You might have noticed that expressing our model through high-level APIs of a deep learning framework requires comparatively few lines of code. We did not have to individually allocate parameters, define our loss function, or implement minibatch stochastic gradient descent. Once we start working with much more complex models, advantages of high-level APIs will grow considerably. However, once we have all the basic pieces in place, the training loop itself is strikingly similar to what we did when implementing everything from scratch.

As a reminder, for some number of epochs, we will make a complete pass over the dataset (`train_data`), iteratively grabbing one minibatch of inputs and the corresponding ground-truth labels. For each minibatch, we go through the following procedure:

* Generate predictions by calling `net(X)` and calculate the loss `l` (the forward propogation)
* Calculate gradients by running the backpropagation
* Update the model parameters by invoking our optimiser

For good measure, we compute the loss after each epoch and print it to monitor our progress.

In [39]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000217
epoch 2, loss 0.000090
epoch 3, loss 0.000090


Below, we compare the model parameters learned by training on finite data and the actual parameters that generated our dataset. To access parameters, we first access the layer that we need from `net` and then access that layer's weights and bias. AS in our from-scratch implementation, note that our estimated parameters are close to their ground-truth counterparts.

In [40]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([-0.0006, -0.0003])
error in estimating b: tensor([0.0002])


### 3.3.8. Summary

* Using PyTorch's high-level APIs, we can implement models much more concisely.
* In PyTorch, the `data` module provides tools for data processing, the `nn` module defines a large number of neural network layers and common loss functions
* We can initialise the parameters by replacing their values with methods ending with `_`